# Part 1: Cleaning and cannonizing DB


Project: Solar Power for Affordable Housing through Computational Design of Low-Cost/High-Efficiency Solar Cells.

Author: AlvaroVM [https://alvarovm.github.io](http://alvarovm.github.io)

Version: 0.0.1

In [1]:
import sys
import os
SRC_DIR='../..'

In [2]:
sys.path.append(os.path.join(SRC_DIR, 'code'))
import utils

../../code/utils.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
#https://github.com/jmcarpenter2/swifter
#import swifter
#2-TSNE-UMAP-map-cuda-Copy1

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs 
from rdkit.Chem import Draw
from rdkit.Chem.rdMolDescriptors import  GetHashedMorganFingerprint
from rdkit.DataStructs import ConvertToNumpyArray

utils.plot_settings2()

results_path = os.path.join(SRC_DIR,'results')

### Tools to Cannonize smiles

In [4]:
# from rdkit.Chem import MolFromSmiles as smi2mol
# from rdkit.Chem.rdMolDescriptors import  GetHashedMorganFingerprint,GetMorganFingerprintAsBitVect
# from rdkit.DataStructs import ConvertToNumpyArray

from rdkit.Chem import MolFromSmiles as smi2mol
from rdkit.Chem.rdMolDescriptors import  GetHashedMorganFingerprint,GetMorganFingerprintAsBitVect
from rdkit.DataStructs import ConvertToNumpyArray

import re

def canon_smiles(smi):
    try:
        m = smi2mol(smi)
    except:
        m = False
        print('hola cannon'+smi)
        
    if m is False:
        return False
    else:
        try:
            sim = Chem.MolToSmiles(m, isomericSmiles=True, canonical=True)
        except:
            sim = False
            #print('hola cannon2'+smi)
        return sim
    

def CleanSMI(smi):
        try: 
            clean=re.sub(r'[<>%\\/?\|]+', '', smi)
        except:
            #print('holaclean ' +smi )
            clean =False
        return clean

def HardValidSMI(smi):
    """
    A rule based function to validate a given smile string. 
    Return type: Boolean
    True: If a match is found. 
    False: Charges, Ions and No Conjugated regions found.
    """

    mysmile = CleanSMI(smi)

    if mysmile is not False:

        try: 
            illegalstring = re.search(r'\\|/|\*|Fe|\+\+|\.|\|',mysmile) #--> Sanity check!
        except:
            #print('holaill ' +smi )
            illegalstring = True
    else:
        return False
    
    if illegalstring:
        return False
    else:
        cansmile = canon_smiles(mysmile)
        if cansmile is False:
            return False
        match = re.search(r'\[\w{1,3}[\+-\.\d]+\]|\[\w{2}\]|\.|\(\*\)',cansmile)
        if match:
            return False
        else:
            conjuated = re.search(r'[a-z\W]\d+[a-zD-Z\W]+\d',mysmile) #r'[a-z\W]\d+[\w\W]+\d' or use (r'[a-z\W]\d+[a-zD-Z\W]+\d',mysmile) )
            if conjuated:
                return True
            else:
                return False


def applyMorganFP(m,**kwargs):
    fptype='bit'
    
    if 'fptype' in kwargs:
        fptype=kwargs['fptype']
    if 'fp_args' in kwargs:
        fp_args=kwargs['fp_args']     
    #fp_args = self.meta_data['fp_args']
    #fptype = self.meta_data['fptype']
    arr = np.zeros((1,))
    if fptype == 'bit': 
        arr = np.zeros((1,))
        #ConvertToNumpyArray(GetHashedMorganFingerprint(m, **fp_args), arr)
        try:
            arr = np.array(GetMorganFingerprintAsBitVect(m, **fp_args))
        except:
            print(Chem.MolToSmiles(m))
    elif fptype == 'count':
        #arr = np.zeros((1,))
        ConvertToNumpyArray(GetHashedMorganFingerprint(m, **fp_args), arr)
    return arr

# from rdkit.Chem import MolFromSmiles as smi2mol


## Open Database

Our database is expresed a CSV file (spread sheet like). This each row has information of the molecular description of in SMILE format, and optical properties as the wavelenght (lamnda/nm), extinction coefficient, first excitation energy with quantum mechanics, etc.

In [5]:
df = pd.read_csv('../../data/extended_db_Zindo_Nov_2019_V5.csv').fillna(value = 0)
print('Column names: {}'.format(str(df.columns.tolist())))
print('Table Shape: {}'.format(df.shape))

Column names: ['inchikey', 'smi_pre', 'smi_post', 'lambda_sTDA (nm)', 'f1_sTDA', 'lumo_dft', 'homo_dft', 'dmom_dft (D)', 'lambda_z (nm)', 'f1_z', 'lumo_z', 'homo_z', 'dmom_z (D)', 'lumo_mopac', 'homo_mopac', 'dmom_mopac (D)', 'lambda_tddft (nm)', 'f1_tddft', 'lambda_exp_max (nm)', 'epsilon_exp_max ', 'lambda_exp_min (nm)', 'epsilon_exp_min ', 'solvent']
Table Shape: (9879, 23)


In [43]:
#df.count()

Unfortunately some of the SMILE that define the molecules are not valid. We have a function called `HardValidSMI` which validate whether the SMILES can be processed. See example

In [6]:
#smi=df['smi_post'].iloc[0]
smi = 'O=C1O[C@H]([C@@H](O)CO)C(O)=C1O'
print(f'Q: Is this {smi} a valid molecule? A: {HardValidSMI(smi)}')

Q: Is this O=C1O[C@H]([C@@H](O)CO)C(O)=C1O a valid molecule? A: False


Remove metals and invalid smiles. Adding a new column `nogood`. Notice that there two SMILES: `smile_pre` and `smile_post`. The `smile_pre` are the SMILES generated directly from the molecules found in the literature, and `smiles_post` are a reduced version with only the active region of the molecule.

## Apply HardValidSMI to columns with SMILES

In [7]:
df['nogood']=df['smi_pre'].apply(HardValidSMI)
#df=df[df['nogood'] != False]
df['nogoodpost']=df['smi_post'].apply(HardValidSMI)
#df=df[df['nogoodp'] != False]

[15:22:19] WARNING: not removing hydrogen atom without neighbors
[15:22:21] WARNING: not removing hydrogen atom without neighbors
[15:22:22] SMILES Parse Error: ring closure 1 duplicates bond between atom 46 and atom 47 for input: 'C(CN1c2cc(ccc2C(=C2C(=O)N(c3c2ccc(c3)c2ccc(cc2)C=C(C(=O)O)C#N)CC(C)CC)C1=O)c1ccc2c(c1)C1C1N2c1ccc(cc1)C)CC'
[15:22:23] SMILES Parse Error: ring closure 4 duplicates bond between atom 19 and atom 20 for input: 'CO[C@H]1C=CO[C@@]2(C)Oc3c(C2=O)c2C(=O)C(=CNN4C4)C(=C(c2c(c3C)O)O)NC(=O)C(=CC=C[C@@H]([C@@H]([C@H]([C@H]([C@H]([C@@H]([C@@H]1C)OC(=O)C)C)O)C)O)C)C'
[15:22:23] Explicit valence for atom # 1 C, 6, is greater than permitted
[15:22:23] Explicit valence for atom # 1 C, 6, is greater than permitted
[15:22:23] Explicit valence for atom # 1 C, 5, is greater than permitted
[15:22:23] Explicit valence for atom # 1 C, 5, is greater than permitted
[15:22:23] Explicit valence for atom # 1 C, 5, is greater than permitted
[15:22:23] Explicit valence for atom # 1 C, 6,

Let's filter the rows with `False` values in `nogood`.

In [9]:
df[df['nogood'] != False]

,inchikey,smi_pre,smi_post,lambda_sTDA (nm),f1_sTDA,lumo_dft,homo_dft,dmom_dft (D),lambda_z (nm),f1_z,...,dmom_mopac (D),lambda_tddft (nm),f1_tddft,lambda_exp_max (nm),epsilon_exp_max,lambda_exp_min (nm),epsilon_exp_min,solvent,nogood,nogoodpost
1,WMBWREPUVVBILR-WIYYLYMNSA-N,Oc1cc(O)c2c(c1)O[C@@H]([C@@H](C2)OC(=O)c1cc(O)...,Oc1cc(O)c2c(c1)O[C@@H]([C@@H](C2)OC(=O)c1cc(O)...,263.5,0.013443,0.7857,-8.0182,4.27854,309.496252,0.018574,...,3.70415,0.0,0.0,280.0,0.0,280.0,0.0,0,True,True
3,FLMQUWVILHJCFJ-UHFFFAOYSA-N,Clc1ccc(c(c1)Cl)OCCn1cncc1,Clc1ccc(c(c1)Cl)OCCn1cncc1,245.2,0.013209,1.0193,-8.8676,3.86453,276.011127,0.010132,...,6.51081,0.0,0.0,254.0,0.0,254.0,0.0,0,True,True
6,UGCLEKVSHFQKOD-LLVKDONJSA-N,COC(=O)Cc1ccc(cc1)OC[C@H]1OC1,COC(=O)Cc1ccc(cc1)OC[C@H]1OC1,245.9,0.039422,1.5588,-8.4152,2.38065,280.952183,0.024477,...,2.03847,0.0,0.0,225.0,0.0,225.0,0.0,0,True,True
7,GSDSWSVVBLHKDQ-UHFFFAOYSA-N,CN1CCN(CC1)c1c(F)cc2c3c1OCC(n3cc(c2=O)C(=O)O)C,CN1CCN(CC1)c1c(F)cc2c3c1OCC(n3cc(c2=O)C(=O)O)C,291.8,0.049058,0.0174,-8.3155,10.03869,345.648727,0.000144,...,11.47002,0.0,0.0,820.0,0.0,820.0,0.0,0,True,True
8,GXOAJZBMIZDGCK-TVONUNMFSA-N,OC[C@H]1O[C@@H](N=C=C2CC(=O)c3c4c2nc2cc(Cl)ccc...,OC[C@H]1O[C@@H](N=C=C2CC(=O)c3c4c2nc2cc(Cl)ccc...,345.9,0.072536,-0.9850,-8.4437,3.35184,386.243609,0.051352,...,3.76588,0.0,0.0,943.0,0.0,943.0,0.0,0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9874,SMURFLRAIWQCMM-UHFFFAOYSA-N,N#Cc1c(cc(c2c1c1cccnc1CC2)c1ccccc1)N1CCCCC1,N#Cc1c(cc(c2c1c1cccnc1CC2)c1ccccc1)N1CCCCC1,291.3,0.038060,-0.0526,-8.4226,3.94833,325.247110,0.249783,...,5.27641,0.0,0.0,540.0,0.0,468.0,0.0,0,True,True
9875,IQLDXBXINQMCHI-UHFFFAOYSA-N,N#Cc1nn(nc1N1CCCC1)c1ccc(cc1)Cl,N#Cc1nn(nc1N1CCCC1)c1ccc(cc1)Cl,294.0,0.631244,0.0838,-8.0247,4.64755,361.680859,0.531235,...,4.82195,0.0,0.0,347.0,0.0,347.0,0.0,0,True,True
9876,YTKGZLZRIUJNQI-UHFFFAOYSA-N,O=c1nc2SCCn2c2c1cnc(n2)N1CCCCC1,O=c1nc2SCCn2c2c1cnc(n2)N1CCCCC1,277.0,0.004539,0.3888,-8.3370,9.34625,338.847222,0.000554,...,9.60212,0.0,0.0,332.0,8000.0,332.0,8000.0,0,True,True
9877,HQYAPNWPDBPXTB-NRFANRHFSA-N,COC(=O)c1cc2ccccc2cc1c1cc2ccccc2cc1C(=O)N([C@H...,COC(=O)c1cc2ccccc2cc1c1cc2ccccc2cc1C(=O)N([C@H...,292.7,0.047515,0.1759,-8.0675,2.38643,324.906180,0.014096,...,2.85462,0.0,0.0,335.2,19400.0,223.0,16600.0,0,True,True


In [49]:
print(f" Shape of smiles_pre that did not pass hard check {df[df['nogood'] != False].shape}")
print(f" Shape of smiles_post that did not pass hard check {df[df['nogoodpost'] != False].shape}")

 Shape of smiles_pre that did not pass hard check (6377, 53)
 Shape of smiles_post that did not pass hard check (6377, 53)


### Create a smiles column with cannonized SMILES using the canon_smiles function

In [9]:
#del df['nogood']
df['smiles'] = df['smi_post'].apply(canon_smiles)
df = df[df['smiles'] != False]
#del df['smi_post']
#df.mask(df.eq('None')).dropna()
#df.head(1)
print('Column names: {}'.format(str(df.columns.tolist())))
print('Table Shape: {}'.format(df.shape))

[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:48] WARNING: not removing hydrogen atom without neighbors
[15:22:49] WARNING: not removing hydrogen atom without neighbors
[15:22:50] SMILES Parse Error: ring closure 1 duplicates bond between atom 46 and atom 47 for input: 'C(CN1c2cc(ccc2C(=C2C(=O)N(c3c2ccc(c3)c2ccc(cc2)C=C(C(=O)O)C#N)CC(C)CC)C1=O)c1ccc2c(c1)C1C1N2c1ccc(cc1)C)CC'
[15:22:50]

Column names: ['inchikey', 'smi_pre', 'smi_post', 'lambda_sTDA (nm)', 'f1_sTDA', 'lumo_dft', 'homo_dft', 'dmom_dft (D)', 'lambda_z (nm)', 'f1_z', 'lumo_z', 'homo_z', 'dmom_z (D)', 'lumo_mopac', 'homo_mopac', 'dmom_mopac (D)', 'lambda_tddft (nm)', 'f1_tddft', 'lambda_exp_max (nm)', 'epsilon_exp_max ', 'lambda_exp_min (nm)', 'epsilon_exp_min ', 'solvent', 'nogood', 'nogoodpost', 'smiles']
Table Shape: (9870, 26)


[15:22:50] Explicit valence for atom # 12 C, 5, is greater than permitted


#### Keep only the rows with lambda_sTDA (nm) that are bigger than zero (disabled)

In [10]:
#df['nostda']=df['lambda_sTDA (nm)'].apply(np.isnan)
#df = df[df.nostda == False]
df = df[df['lambda_sTDA (nm)'] >0 ]
df.shape

(7177, 26)

### Move smiles columns front

In [11]:
cols= list(df)
cols.insert(0,cols.pop(cols.index('smiles')))
df=df[cols]
print('Column names: {}'.format(str(df.columns.tolist())))

Column names: ['smiles', 'inchikey', 'smi_pre', 'smi_post', 'lambda_sTDA (nm)', 'f1_sTDA', 'lumo_dft', 'homo_dft', 'dmom_dft (D)', 'lambda_z (nm)', 'f1_z', 'lumo_z', 'homo_z', 'dmom_z (D)', 'lumo_mopac', 'homo_mopac', 'dmom_mopac (D)', 'lambda_tddft (nm)', 'f1_tddft', 'lambda_exp_max (nm)', 'epsilon_exp_max ', 'lambda_exp_min (nm)', 'epsilon_exp_min ', 'solvent', 'nogood', 'nogoodpost']


In [17]:
#max(df.smiles.apply(len))

### Work only with post trimming

In [12]:
df[df['lambda_sTDA (nm)']>0].count()

smiles                 7177
inchikey               7177
smi_pre                7177
smi_post               7177
lambda_sTDA (nm)       7177
f1_sTDA                7177
lumo_dft               7177
homo_dft               7177
dmom_dft (D)           7177
lambda_z (nm)          7177
f1_z                   7177
lumo_z                 7177
homo_z                 7177
dmom_z (D)             7177
lumo_mopac             7177
homo_mopac             7177
dmom_mopac (D)         7177
lambda_tddft (nm)      7177
f1_tddft               7177
lambda_exp_max (nm)    7177
epsilon_exp_max        7177
lambda_exp_min (nm)    7177
epsilon_exp_min        7177
solvent                7177
nogood                 7177
nogoodpost             7177
dtype: int64

### Create column with a RDKIT molecule object using the SMILES

In [13]:
df['mol'] = df['smiles'].apply(Chem.MolFromSmiles)

In [15]:
print('Column names: {}'.format(str(df.columns.tolist())))

Column names: ['smiles', 'inchikey', 'smi_pre', 'smi_post', 'lambda_sTDA (nm)', 'f1_sTDA', 'lumo_dft', 'homo_dft', 'dmom_dft (D)', 'lambda_z (nm)', 'f1_z', 'lumo_z', 'homo_z', 'dmom_z (D)', 'lumo_mopac', 'homo_mopac', 'dmom_mopac (D)', 'lambda_tddft (nm)', 'f1_tddft', 'lambda_exp_max (nm)', 'epsilon_exp_max ', 'lambda_exp_min (nm)', 'epsilon_exp_min ', 'solvent', 'nogood', 'nogoodpost', 'mol']


### Create column with the Morgan fingerprint using the applyMorganFP to create a 2048 bit vectors using fragments up to radii 6, for example: fp= applyMorganFP(mol,fptype='bit',fp_args={'radius':6, 'nBits':2048})

In [16]:
df['morganfps-b']=df['mol'].apply(applyMorganFP,fptype='bit',fp_args={'radius':6, 'nBits':2048})
#df['morganfps-b8']=df['mol'].apply(applyMorganFP,fptype='bit',fp_args={'radius':8, 'nBits':2048})

### Compute a set of descriptors and add them as extra columns to the Dataframe

In [17]:
#dfc.shape[0]
fpfail=[]
#for e in dfc.shape[0]:
for e in range(df.shape[0]):
    try:
        fp = applyMorganFP(df.iloc[e].mol,fptype='count',fp_args={'radius':4, 'nBits':2048})
        fpfail.append(True)
    except:
        fpfail.append(False)
        print(e)
    
#applyMorganFP(e,fptype='count',fp_args={'radius':4, 'nBits':2048})

df.insert(2,'fpfail',fpfail,True)

df = df[df.fpfail!=False]
df.shape

(7177, 29)

In [18]:
getonly=['NHOHCount', 'NOCount', 'NumAliphaticCarbocycles',
                           'NumAliphaticHeterocycles', 'NumAliphaticRings',
                           'NumAromaticCarbocycles', 'NumAromaticHeterocycles',
                           'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 
                           'NumHeteroatoms', 'NumRadicalElectrons', 'NumRotatableBonds',
                           'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 
                           'NumSaturatedRings', 'NumValenceElectrons',
                            ]

In [19]:
import inspect
import rdkit.Chem.Descriptors as Descriptors
#https://github.com/jmcarpenter2/swifter
#import swifter 
from collections import OrderedDict

calc_props = OrderedDict(inspect.getmembers(Descriptors, inspect.isfunction))
for key in list(calc_props.keys()):
    if key.startswith('_'):
        del calc_props[key]
    else:
        thisnot=False
        for myprop in getonly:
            if myprop == key:
                thisnot=True
        if not thisnot:
            del calc_props[key]

In [20]:
from tqdm import tqdm
def calc_all(dfc,calc_props,smiles_col='smiles'):
    #df['mol'] = df[smiles_col].apply(Chem.MolFromSmiles)
    for key,val in tqdm (calc_props.items()):
    #for key,val in calc_props.items():
        #df[key] = df['mol'].apply(val)
        df[key] = df['mol'].apply(val)
    return df
print('Found {} molecular descriptors in RDKIT'.format(len(calc_props)))
#calc_props

Found 17 molecular descriptors in RDKIT


In [21]:
df=calc_all(df,calc_props)

100%|██████████| 17/17 [00:02<00:00,  7.47it/s]


In [22]:
print('Column names: {}'.format(str(df.columns.tolist())))

Column names: ['smiles', 'inchikey', 'fpfail', 'smi_pre', 'smi_post', 'lambda_sTDA (nm)', 'f1_sTDA', 'lumo_dft', 'homo_dft', 'dmom_dft (D)', 'lambda_z (nm)', 'f1_z', 'lumo_z', 'homo_z', 'dmom_z (D)', 'lumo_mopac', 'homo_mopac', 'dmom_mopac (D)', 'lambda_tddft (nm)', 'f1_tddft', 'lambda_exp_max (nm)', 'epsilon_exp_max ', 'lambda_exp_min (nm)', 'epsilon_exp_min ', 'solvent', 'nogood', 'nogoodpost', 'mol', 'morganfps-b', 'NHOHCount', 'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles', 'NumAliphaticRings', 'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRadicalElectrons', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 'NumSaturatedRings', 'NumValenceElectrons']


### Compute the energy GAP (LUMO-HOMO) from mopac, zindo and dft, compare them.

In [23]:
df['gapdft']=df['lumo_dft']-df['homo_dft']
#dfzi.homo_z= dfzi.homo_z.apply(float)
#dfzi['gapz']=dfzi['lumo_z']-dfzi['homo_z']
#dfzi['gapmopac']=dfzi['lumo_mopac']-dfzi['homo_mopac']

In [24]:
df.homo_z = df.homo_z.apply(float)
df['gapz'] = df['lumo_z'] - df['homo_z']

In [25]:
df['gapmopac']=df['lumo_mopac']-df['homo_mopac']

### Save the Dataframe as a pickle file

In [26]:
df.to_pickle('../../data/extended_db_Zindo_Nov_2019_V5_cannfp.pkl')
print('Column names: {}'.format(str(df.columns.tolist())))
print('Table Shape: {}'.format(df.shape))

Column names: ['smiles', 'inchikey', 'fpfail', 'smi_pre', 'smi_post', 'lambda_sTDA (nm)', 'f1_sTDA', 'lumo_dft', 'homo_dft', 'dmom_dft (D)', 'lambda_z (nm)', 'f1_z', 'lumo_z', 'homo_z', 'dmom_z (D)', 'lumo_mopac', 'homo_mopac', 'dmom_mopac (D)', 'lambda_tddft (nm)', 'f1_tddft', 'lambda_exp_max (nm)', 'epsilon_exp_max ', 'lambda_exp_min (nm)', 'epsilon_exp_min ', 'solvent', 'nogood', 'nogoodpost', 'mol', 'morganfps-b', 'NHOHCount', 'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles', 'NumAliphaticRings', 'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRadicalElectrons', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 'NumSaturatedRings', 'NumValenceElectrons', 'gapdft', 'gapz', 'gapmopac']
Table Shape: (7177, 49)
